In [3]:
import sys
import pandas as pd
import os
import re
import glob
import shutil
import numpy as np
from pathlib import Path
import shutil
from sklearn.calibration import CalibratedClassifierCV

from IPython.core.display import display, HTML
# make the Jupyter notebook use the full screen width
display(HTML("<style>.container { width:99% !important; }</style>"))


In [4]:
import glob
import json
import collections
from base64 import b64encode
from base64 import b64decode

#DATA_DIR= "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/EUROLEX/Raw"
DATA_DIR= "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_28_04"


Eurolex_files=glob.glob( os.path.join( DATA_DIR, "EURLEX" , "*jsonl" )  )

BIS_files=glob.glob( os.path.join( DATA_DIR, "BIS" , "*jsonl" )  )

EIOPA_files=glob.glob( os.path.join( DATA_DIR, "EIOPA" , "*jsonl" )  )

FSB_files=glob.glob( os.path.join( DATA_DIR, "FSB" , "*jsonl" )  )

SRB_files=glob.glob( os.path.join( DATA_DIR, "SRB" , "*jsonl" )  )

files=Eurolex_files + BIS_files + EIOPA_files + FSB_files + SRB_files

all_jsons=[]

for file in files:
    jsonl=open( file ).read().rstrip("\n").split( "\n" )
    json_file = [json.loads( json_line ) for json_line in jsonl]
    all_jsons.append( json_file )

    
status=[]
date=[]
title=[]
summary=[]

for json_file in all_jsons:
    metadata=json_file[0]
    if 'status' in metadata.keys():
        status.append( metadata['status'][0]   )
    else: 
        status.append( '-'  )
        
    if 'date' in metadata.keys():
        date.append( metadata['date'][0]   )
    else: 
        date.append( '-'  )
    
    if 'title' in metadata.keys():
        title.append( metadata['title'][0]   )
    else: 
        title.append( '-'  )
        
    if 'summary' in metadata.keys():
        summary.append( metadata['summary'][0]   )
    else: 
        summary.append( '-'  )
        
#only keep title, date and target (i.e. status)    



target=len(status)*['-']
for i,stat in enumerate( status):
    if ("No longer in force" in stat) or ( "Superseded" in stat ) or ( "Modified" in stat ):
        target[i]="declined"
    elif ("In force" in stat) or ("Current" in stat) or ("Consolidated" in stat):
        target[i]='accepted'
    elif ("Date of entry into force unknown" in stat ) or ('-' in stat):
        target[i]='unknown'
    else:
        target[i]='unknown'
                     
print( "Class distribution before cleaning:", collections.Counter(target) )
        
titles_clean=[]
date_clean=[]
target_clean=[]
for tit, dat, tar in zip( title, date, target ):
    if (tar!='unknown') and (dat !='-') and (tit !='-'):
        titles_clean.append( tit )
        date_clean.append(dat)
        target_clean.append(  tar  )

print( "Class distribution after cleaning:", collections.Counter(target_clean) )
  
#Extract the year from the date. 

year=[ int( one_date.split("-")[0] ) for one_date in date_clean ]

#Concatenate the year with the title.

titles_concat=[]
for yea,tit in zip(year,titles_clean):
    titles_concat.append( str( yea)+' '+ tit )

Class distribution before cleaning: Counter({'declined': 9028, 'unknown': 5966, 'accepted': 2383})
Class distribution after cleaning: Counter({'declined': 7289, 'accepted': 2336})


In [16]:
all_jsons[1400]

[{'celex': ['31980R3450'],
  'title': ['Council Regulation (EEC) No 3450/80 of 22 December 1980 laying down certain measures for the conservation and management of fishery resources applicable to vessels flying the flag of certain third countries in the 200 nautical-mile zone off the coast of the French department of Guyana'],
  'status': ['No longer in force, Date of end of validity: 31/03/1981'],
  'ELI': ['http://data.europa.eu/eli/reg/1980/3450/oj'],
  'various': ['OJ L 360, 31.12.1980, p. 7–12 (DA, DE, EN, FR, IT, NL)Greek special edition: Chapter 04 Volume 002 P. 30  - 36, ELI: http://data.europa.eu/eli/reg/1980/3450/oj'],
  'pdf_docs': ['https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:31980R3450&from=EN'],
  'date': ['1980-12-22T00:00:00Z'],
  'dates': ['1980-12-22T00:00:00Z',
   '1981-01-03T00:00:00Z',
   '1981-03-31T00:00:00Z'],
  'dates_type': ['date of document',
   'date of effect',
   'date of end of validity'],
  'dates_info': ['n/a',
   'Entry into force  D

In [32]:
all_jsons[5010]

[{'celex': ['31995D0559'],
  'title': ['95/559/EC, Euratom: Council and Commission Decision of 4 December 1995 concluding the Additional Protocol to the Europe Agreement establishing an association between the European Communities and their Member States, of the one part, and the Republic of Hungary, of the other part'],
  'status': ['In force'],
  'ELI': ['http://data.europa.eu/eli/dec/1995/559/oj'],
  'various': ['OJ L 317, 30.12.1995, p. 29–29 (ES, DA, DE, EL, EN, FR, IT, NL, PT, FI, SV), ELI: http://data.europa.eu/eli/dec/1995/559/oj'],
  'pdf_docs': ['https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:31995D0559&from=EN'],
  'date': ['1995-12-04T00:00:00Z'],
  'dates': ['1995-12-04T00:00:00Z',
   '1995-12-04T00:00:00Z',
   '9999-01-01T00:00:00Z'],
  'dates_type': ['date of document',
   'date of effect',
   'date of end of validity'],
  'dates_info': ['n/a', 'Entry into force  Date of document', 'n/a'],
  'classifications_label': ['Hungary',
   'European integration',
 

In [20]:
all_jsons[1500]

[{'id': '1a27edc1-12e7-5d63-9514-63fec314372e',
  'celex': ['31982R1998'],
  'title': ['Commission Regulation (EEC) No 1998/82 of 22 July 1982 altering the basic amount of the import levy on syrups and certain other sugar products'],
  'type': ['Regulation'],
  'date': ['1982-07-23T00:00:00Z'],
  'url': ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:31982R1998'],
  'eli': ['http://data.europa.eu/eli/reg/1982/1998/oj'],
  'website': ['<span class="highlight">EURLEX</span>'],
  'various': ['OJ L 215, 23.7.1982, p. 33–34 (DA, DE, EL, EN, FR, IT, NL), ELI: http://data.europa.eu/eli/reg/1982/1998/oj'],
  'accepted_probability': [0.0],
  'pull': [False],
  'created_at': ['2020-04-21T12:18:01.045Z'],
  'updated_at': ['2020-04-21T12:18:01.045Z'],
  '_version_': 1664593875872776192}]

In [53]:
all_jsons[10]

[{'celex': ['31982R1886'],
  'title': ['Commission Regulation (EEC) No 1886/82 of 14 July 1982 fixing the import levies on cereals and on wheat or rye flour, groats and meal'],
  'ELI': ['http://data.europa.eu/eli/reg/1982/1886/oj'],
  'various': ['OJ L 207, 15.7.1982, p. 9–10 (DA, DE, EL, EN, FR, IT, NL), ELI: http://data.europa.eu/eli/reg/1982/1886/oj'],
  'pdf_docs': ['https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:31982R1886&from=EN'],
  'date': ['1982-07-15T00:00:00Z'],
  'dates': ['1982-07-15T00:00:00Z'],
  'dates_type': ['date of document'],
  'dates_info': ['n/a'],
  'misc_author': ['European Commission'],
  'type': ['Regulation'],
  'misc_form': ['Regulation'],
  'relationships_treaty': ['Select all documents based on this document'],
  'relationships_oj_link': ['https://eur-lex.europa.eu/legal-content/AUTO/?uri=OJ:L:1982:207:TOC'],
  'url': ['https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:31982R1886'],
  'task': [11],
  'website': ['<span class="high

In [26]:
title_target=[]
for title, target in zip( titles_concat , target_clean ):
    if target == 'accepted':
        label_nr=1
    else:
        label_nr=0
    title_target.append( ( title, target , label_nr  )  )

In [27]:
import random 
random.shuffle(title_target)
title_target

[('1977 77/454/EEC: Council Decision of 27 June 1977 setting up an Advisory Committee on Training in Nursing',
  'accepted',
  1),
 ('1999 Gathering Year 2000 Information from Financial Institutions: Recommendations for Supervisors ',
  'declined',
  0),
 ('1992 Commission Regulation (EEC) No 3838/92 of 28 December 1992 amending Regulation (EEC) No 643/86 laying down detailed rules for the application of the supplementary trade mechanism to the live plants and floriculture products listed in Annex XXII to the Act of Accession and imported into Portugal concerning the target ceilings for the year 1993',
  'declined',
  0),
 ('1982 82/279/EEC: Council Decision of 26 April 1982 on the conclusion of the Agreement in the form of an exchange of letters concerning an interim extension of the Protocol annexed to the Agreement between the Government of the Republic of Guinea Bissau and the European Economic Community on fishing off the coast of Guinea Bissau',
  'declined',
  0),
 ('1989 89/659

In [32]:
len(title_target)
test_set=title_target[:2000]
train_set=title_target[2000:]

In [36]:
output_dir=os.path.join( DATA_DIR, 'processed'  ) 
os.makedirs( output_dir )

In [45]:
with open( os.path.join(  output_dir, "train_data.tsv"  ) , 'w'  ) as f:
    for title, label, label_nr in train_set:
        encoded_title = b64encode( title.encode() )
        f.write( f"{encoded_title.decode()  }\t{ label }\t{ label_nr }\n" )
        
with open( os.path.join(  output_dir, "test_data.tsv"  ) , 'w'  ) as f:
    for title, label, label_nr in test_set:
        encoded_title = b64encode( title.encode() )
        f.write( f"{encoded_title.decode()  }\t{ label }\t{ label_nr }\n" )
        

In [46]:
with open( os.path.join(  output_dir, "train_data.csv"  ) , 'w'  ) as f:
    for title, label, label_nr in train_set:
        f.write( f"\"{ title  }\"￭{ label }￭{ label_nr }\n" )

with open( os.path.join(  output_dir, "test_data.csv"  ) , 'w'  ) as f:
    for title, label, label_nr in test_set:
        f.write( f"\"{ title  }\"￭{ label }￭{ label_nr }\n" )        
        

# END